# --- Day 16: Packet Decoder --- 

https://adventofcode.com/2021/day/16

## Get Input Data

In [1]:
with open('../inputs/BITS.txt') as file:
    BITS = file.readline().strip()
BITS

'40541D900AEDC01A88002191FE2F45D1006A2FC2388D278D4653E3910020F2E2F3E24C007ECD7ABA6A200E6E8017F92C934CFA0E5290B569CE0F4BA5180213D963C00DC40010A87905A0900021B0D624C34600906725FFCF597491C6008C01B0004223342488A200F4378C9198401B87311A0C0803E600FC4887F14CC01C8AF16A2010021D1260DC7530042C012957193779F96AD9B36100907A00980021513E3943600043225C1A8EB2C3040043CC3B1802B400D3CA4B8D3292E37C30600B325A541D979606E384B524C06008E802515A638A73A226009CDA5D8026200D473851150401E8BF16E2ACDFB7DCD4F5C02897A5288D299D89CA6AA672AD5118804F592FC5BE8037000042217C64876000874728550D4C0149F29D00524ACCD2566795A0D880432BEAC79995C86483A6F3B9F6833397DEA03E401004F28CD894B9C48A34BC371CF7AA840155E002012E21260923DC4C248035299ECEB0AC4DFC0179B864865CF8802F9A005E264C25372ABAC8DEA706009F005C32B7FCF1BF91CADFF3C6FE4B3FB073005A6F93B633B12E0054A124BEE9C570004B245126F6E11E5C0199BDEDCE589275C10027E97BE7EF330F126DF3817354FFC82671BB5402510C803788DFA009CAFB14ECDFE57D8A766F0001A74F924AC99678864725F253FD134400F9B5D3004A46489A00A4BEAD8F7F1F7497C

## Part 1
---

In [2]:
def hex_to_bin(hex_str):
    "Take a string of hexadecimal characters and convert each to a 4 character binary string."

    bin_str = ''

    for char in hex_str:
        bin_str += bin(int(char, 16))[2:].zfill(4)  # left pad with zeros so len(bin_str) == 4

    return bin_str

#### Run some tests

In [3]:
hex_to_bin('0')  # Should return '0000'

'0000'

In [4]:
hex_to_bin('5')  # Should return '0101'

'0101'

In [5]:
hex_to_bin('f')  # Should return '1111'

'1111'

In [6]:
hex_to_bin('D2FE28') == '110100101111111000101000'

True

In [7]:
hex_to_bin('38006F45291200') == '00111000000000000110111101000101001010010001001000000000'

True

In [8]:
hex_to_bin('EE00D40C823060') == '11101110000000001101010000001100100000100011000001100000'

True

In [9]:
def get_header_info(bin_str, position):
    """Extract the packet header info.
    Return integer representation of packet version, and packet type, and position.
    """

    version = int(bin_str[position : position + 3], base=2)
    type = int(bin_str[position + 3 : position + 6], base=2)

    # Move position to next place in bin_str
    position = position + 6

    return version, type, position

#### Run a test

In [10]:
get_header_info(hex_to_bin('D2FE28'), 0)  # Should return 6, 4, 6

(6, 4, 6)

In [11]:
def decode_packet(packet, version_sum, position):
    """Decode a transmission packet."""

    version, type_id, position = get_header_info(packet, position)
    version_sum += version

    # Literal value packets -- stopping condition, essentially
    if type_id == 4:
        value, position = decode_literal_value(packet, position)

    # Operator packets -- can contain subpackets, so recurssive calls to decode_packet() are made.
    else:
        version_sum, position = decode_operator_packet(packet, position, version_sum)

    return version_sum, position

In [12]:
def decode_literal_value(bin_str, position):
    """Decode literal value packet."""

    # Build up binary string of the literal value
    value_bin = ''

    while bin_str[position] != '0':
        value_bin += bin_str[(position + 1) : (position + 5)]
        position += 5

    # Get last binary string group after bin_str[position] == 0
    value_bin += bin_str[(position + 1) : (position + 5)]

    # Convert from bindary string to integer
    value = int(value_bin, base=2)

    # And increment the position past the last group, too
    position += 5

    return value, position

#### Run a test

In [13]:
decode_literal_value(hex_to_bin('D2FE28'), 6)  # Should return 2021, 21

(2021, 21)

In [14]:
decode_packet(hex_to_bin('D2FE28'), 0, 0)  # Should return 6, 21

(6, 21)

In [15]:
def decode_operator_packet(bin_str, position, version_sum):
    """Decode operater packet, which can contain multiple subpackets."""

    # Length type ID == '0' means next 15 bits represent
    # the total length (in bits) of the subpackets contained in this packet
    if bin_str[position] == '0':

        subpacket_length = int(bin_str[(position + 1) : (position + 16)], base=2)
        position += 16
        end_subpackets = position + subpacket_length
        
        while position < end_subpackets:
            version_sum, position = decode_packet(bin_str, version_sum, position)

    # Lenght type ID == '1' means next 11 bits represent
    # the number of subpackets contained in this packet
    elif bin_str[position] == '1':

        num_subpackets = int(bin_str[(position + 1) : (position + 12)], base=2)
        position += 12
        for _ in range(num_subpackets):
            version_sum, position = decode_packet(bin_str, version_sum, position)

    return version_sum, position

#### Run some tests

**Example 1:** hex string => `38006F45291200`

In [16]:
example_bin_str = hex_to_bin('38006F45291200')
example_bin_str

'00111000000000000110111101000101001010010001001000000000'

In [17]:
get_header_info(example_bin_str, 0)[0]  # Should return 1

1

In [18]:
# Version for subpacket A:
get_header_info('11010001010', 0)[0]

6

In [19]:
# Version for subpacket B:
get_header_info('0101001000100100', 0)[0]

2

In [20]:
decode_packet(hex_to_bin('38006F45291200'), 0, 0)[0]  # Should return 9 (1 + 6 + 2)

9

**Example 2:** hex string => `EE00D40C823060`

In [21]:
example2_bin_str = hex_to_bin('EE00D40C823060')
example2_bin_str

'11101110000000001101010000001100100000100011000001100000'

In [22]:
get_header_info(example2_bin_str, 0)[0]  # Should return 7

7

In [23]:
# Version for subpacket A
get_header_info('01010000001', 0)[0]

2

In [24]:
# Version for subpackat B
get_header_info('10010000010', 0)[0]

4

In [25]:
# Version for subpacket C
get_header_info('00110000011', 0)[0]

1

In [26]:
decode_packet(hex_to_bin('EE00D40C823060'), 0, 0)[0]  # Should return 14 (7 + 2 + 4 + 1)

14

### Run on Test Data

In [27]:
decode_packet(hex_to_bin('8A004A801A8002F478'), 0, 0)[0]  # Should return 16

16

In [28]:
decode_packet(hex_to_bin('620080001611562C8802118E34'), 0, 0)[0]  # Should return 12

12

In [29]:
decode_packet(hex_to_bin('C0015000016115A2E0802F182340'), 0, 0)[0]  # Should return 23

23

In [30]:
decode_packet(hex_to_bin('A0016C880162017C3686B18A3D4780'), 0, 0)[0]  # Should return 31

31

### Run on Input Data

In [31]:
decode_packet(hex_to_bin(BITS), 0, 0)[0] 

945

## Part 2
---

In [109]:
def decode_packet2(packet, position, values):
    """Decode a transmission packet."""

    _, type_id, position = get_header_info(packet, position)

    # Literal value packets
    if type_id == 4:
        value, position = decode_literal_value2(packet, position)
        values.append(value)

    # Operator packets
    elif type_id == 0:
        values, position = decode_operator_packet2(packet, position, values)

    elif type_id == 1:
        values, position = decode_operator_packet2(packet, position, values)

    elif type_id == 2:
        values, position = decode_operator_packet2(packet, position, values)

    elif type_id == 3:
        values, position = decode_operator_packet2(packet, position, values)

    elif type_id == 5:
        values, position = decode_operator_packet2(packet, position, values)

    elif type_id == 6: 
        values, position = decode_operator_packet2(packet, position, values)

    elif type_id == 7:
        values, position = decode_operator_packet2(packet, position, values)

    return values, position

In [110]:
def decode_literal_value2(bin_str, position):
    """Decode literal value packet."""

    # Build up binary string of the literal value
    value_bin = ''

    while bin_str[position] != '0':
        value_bin += bin_str[(position + 1) : (position + 5)]
        position += 5

    # Get last binary string group after bin_str[position] == 0
    value_bin += bin_str[(position + 1) : (position + 5)]

    # Convert from bindary string to integer
    value = int(value_bin, base=2)

    # And increment the position past the last group, too
    position += 5

    return value, position

In [111]:
def decode_operator_packet2(bin_str, position, values):
    """Decode operater packet, which can contain multiple subpackets."""

    # Length type ID == '0' means next 15 bits represent
    # the total length (in bits) of the subpackets contained in this packet
    if bin_str[position] == '0':

        subpacket_length = int(bin_str[(position + 1) : (position + 16)], base=2)
        position += 16
        end_subpackets = position + subpacket_length
        
        while position < end_subpackets:
            values, position = decode_packet2(bin_str, position, values)

    # Lenght type ID == '1' means next 11 bits represent
    # the number of subpackets contained in this packet
    elif bin_str[position] == '1':

        num_subpackets = int(bin_str[(position + 1) : (position + 12)], base=2)
        position += 12
        for _ in range(num_subpackets):
            values, position = decode_packet2(bin_str, position, values)

    return values, position

### Run on Test Data

In [112]:
decode_packet2(hex_to_bin('C200B40A82'), 0, [])  # Should return 3, because 1 + 2 == 3

([1, 2], 40)

In [113]:
decode_packet2(hex_to_bin('04005AC33890'), 0, [])  # Should return 54, because 6 * 9 == 54

([6, 9], 44)

In [114]:
decode_packet2(hex_to_bin('880086C3E88112'), 0, [])  # Should return 7, because 7 == min(7, 8, 9)

([7, 8, 9], 55)

In [116]:
decode_packet2(hex_to_bin('CE00C43D881120'), 0, [])  # Should return 9, because 9 == max(7, 8, 9)

([7, 8, 9], 51)

In [117]:
decode_packet2(hex_to_bin('D8005AC2A8F0'), 0, [])  # Should return 1, because 5 < 15 => True

([5, 15], 44)

In [118]:
decode_packet2(hex_to_bin('F600BC2D8F'), 0, [])  # Should return 0, because 5 > 15 => False

([5, 15], 40)

In [119]:
decode_packet2(hex_to_bin('9C005AC2F8F0'), 0, [])  # Should return 0, because 5 == 15 => False

([5, 15], 44)

In [121]:
decode_packet2(hex_to_bin('9C0141080250320F1802104A08'), 0, []) # Should return 1, because 1 + 3 == 2 * 2

([1, 3, 2, 2], 102)

### Run on Input Data